# Building optimization models from pandas dataframes

`gurobipy-pandas`: The missing link between `pandas` and `gurobipy`!

Key points:

- Pandas style is to avoid writing explicit loops and instead do high level declarative operations. This library enables that style for writing optimization models.
- This is not the only way to write Gurobi models using pandas data.
- This library does not guarantee super-fast code. Rather, following the style enabled by this library should avoid some of the major performance bugs we've seen in user code that mixes gurobipy and pandas.
- You should prepare your data in advance, to match your mathematical model. There are nice crossovers between pandas structures and the parts of an optimization model, which you can exploit to produce clear code without unnecessary overhead.

Todo:

- Try to disambiguate symbols (use more of the alphabet to avoid redefining, x, y, z, c, etc)
- Review the order of expression building examples (should build up to the difficult result)

In this session:

- Overview of `gurobipy-pandas` design
- Basic usage mechanics
    - Creating variables
    - Building expressions using series of variables and data
    - Creating constraints
    - Retrieving solutions
- A complete modelling example

# Design Principles

- Optimization models define all data, variables, and constraints over indexes

$$
\begin{alignat}{2}
\max \quad        & \sum_{i \in I} \sum_{j \in J} p_{ij} x_{ij} \\
\mbox{s.t.} \quad & \sum_{i \in J_i} w_{ij} x_{ij} \le c_{i} & \forall i \in I \\
                  & x_{ij} \in \lbrace 0, 1 \rbrace & \forall i \in I, j \in J_i \\
\end{alignat}
$$

- The mathematical indices provide a clear way to structure data in code

In [1]:
# Just some random data to show.
import pandas as pd
import numpy as np
product_data = pd.DataFrame(
    index=pd.MultiIndex.from_tuples([(1, 2), (1, 1), (2, 3), (2, 1), (3, 1)]),
    columns=["cost_per_unit", "amount_available"],
    data=np.random.random((5, 2)).round(2)
)

# Design Principles

- Pandas DataFrames and Series *also* define data over indexes

In [2]:
product_data

cost_per_unit  amount_available
1 2           0.93              0.76
  1           0.56              0.98
2 3           0.12              0.83
  1           0.71              0.76
3 1           0.75              0.58

- We just need a way to define variables and build constraints over those indexes

# Missing Link?

`gurobipy-pandas` provides:

- Methods to create pandas-indexed series of variables
- Methods to build constraints from expressions
- Accessors to extract solutions as pandas structures

`pandas` provides:

- Existing algebra/split-apply-combine logic (well known transforms)

- With pandas, we are already set up to define all our data over various indexes, aligned with the mathematical model.
- What we are missing:
    - Easy way to define/construct variables over the same indexes using pandas structures
    - Methods to transform data and variables into constraint expressions (actually, we get this for free)
    - Easy way to take the resulting expressions and build constraints from them

# Installation

```
pip install gurobipy-pandas
```

Then, add one more import to your usual arsenal:

In [3]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

import gurobipy_pandas as gppd

# Handy trick for live coding
gppd.set_interactive()

# Silence please
gp.setParam('OutputFlag', 0)

# Usage

- `gurobipy` objects are still the entry point for:
    - creating models
    - starting optimization
    - constants, status codes, etc
- `gurobip-pandas` provides accessors and functions to:
    - create sets of variables based on indexes
    - create constraints based on aligned series
    - extract solutions as series

# Creating Models

The usual way; entry-point doesn't change

In [4]:
model = gp.Model()

# Creating Variables

- Using free functions

- Free function: returns a series of variables (not part of a dataframe)
- Variables live in the namespace, not as columns

In [5]:
data = pd.DataFrame(
    {
        "i": [0, 0, 1, 2, 2],
        "j": [1, 2, 0, 0, 1],
        "p": [0.3, 1.2, 0.7, 0.9, 1.2],
        "w": [1.3, 1.7, 1.4, 1.1, 0.9],
    }
).set_index(["i", "j"])

In [6]:
data

p    w
i j          
0 1  0.3  1.3
  2  1.2  1.7
1 0  0.7  1.4
2 0  0.9  1.1
  1  1.2  0.9

In [7]:
x = gppd.add_vars(model, data, name="x", obj="p")
x

i  j
0  1    <gurobi.Var x[0,1]>
   2    <gurobi.Var x[0,2]>
1  0    <gurobi.Var x[1,0]>
2  0    <gurobi.Var x[2,0]>
   1    <gurobi.Var x[2,1]>
Name: x, dtype: object

# Creating Variables

- Using dataframe accessors

- Dataframe accessor: returns a new dataframe with an appended column
- Use this approach to allow method chaining
- Variables live as columns in a dataframe, along with other data/objects sharing that index

In [8]:
data

p    w
i j          
0 1  0.3  1.3
  2  1.2  1.7
1 0  0.7  1.4
2 0  0.9  1.1
  1  1.2  0.9

In [9]:
variables = (
    data
    .gppd.add_vars(model, name="y", ub=1.0)
    .gppd.add_vars(model, name="z", obj="p")
)
variables

p    w                    y                    z
i j                                                    
0 1  0.3  1.3  <gurobi.Var y[0,1]>  <gurobi.Var z[0,1]>
  2  1.2  1.7  <gurobi.Var y[0,2]>  <gurobi.Var z[0,2]>
1 0  0.7  1.4  <gurobi.Var y[1,0]>  <gurobi.Var z[1,0]>
2 0  0.9  1.1  <gurobi.Var y[2,0]>  <gurobi.Var z[2,0]>
  1  1.2  0.9  <gurobi.Var y[2,1]>  <gurobi.Var z[2,1]>

# Creating Expressions

- Pandas handles this for us
- We always leverage pandas-native functions
- Common operations in mathprog:
    - summation
    - arithmetic operations
    - groupby (split-apply-combine) operations
- Let's explore some common mathematical expressions

Summation over a single index:

$$
\sum_i x_i
$$

In [10]:
i = pd.RangeIndex(5)
x = gppd.add_vars(model, i, name="x")
x

0    <gurobi.Var x[0]>
1    <gurobi.Var x[1]>
2    <gurobi.Var x[2]>
3    <gurobi.Var x[3]>
4    <gurobi.Var x[4]>
Name: x, dtype: object

In [11]:
x.sum()

<gurobi.LinExpr: x[0] + x[1] + x[2] + x[3] + x[4]>

Scalar multiplication:

$$
2 x_i
$$

In [12]:
2*x

0    2.0 x[0]
1    2.0 x[1]
2    2.0 x[2]
3    2.0 x[3]
4    2.0 x[4]
Name: x, dtype: object

Series-wise multiplication:

$$
c_i x_i
$$

In [13]:
c = pd.Series(index=i, name="c", data=np.arange(1, 6))
c * x

0        x[0]
1    2.0 x[1]
2    3.0 x[2]
3    4.0 x[3]
4    5.0 x[4]
dtype: object

Summing the result:

$$
\sum_i c_i x_i
$$

In [14]:
(c * x).sum()

<gurobi.LinExpr: x[0] + 2.0 x[1] + 3.0 x[2] + 4.0 x[3] + 5.0 x[4]>

## Comfortable/Familiar?

Hopefully!

Any operation you would do with data in pandas, you can do in the same way with data & variables.

Let's look at some multi-index cases.

For each $i$, sum over all corresponding $j$:

$$
\sum_{i \in I} x_{ij} \quad \forall j \in J
$$

In [15]:
x = gppd.add_vars(model, data, name="x")
x

i  j
0  1    <gurobi.Var x[0,1]>
   2    <gurobi.Var x[0,2]>
1  0    <gurobi.Var x[1,0]>
2  0    <gurobi.Var x[2,0]>
   1    <gurobi.Var x[2,1]>
Name: x, dtype: object

In [16]:
x.groupby("i").sum()

i
0        x[0,1] + x[0,2]
1    <gurobi.Var x[1,0]>
2        x[2,0] + x[2,1]
Name: x, dtype: object

Correspondence:

- index specified in `groupby` matches `\forall`
- `.sum()` reduces over the remaining index levels

Align data on partial indexes:

$$
c_i x_{ij}
$$

In [17]:
c = pd.Series(index=pd.RangeIndex(3, name='i'), data=[1, 2, 3], name="c")
c

i
0    1
1    2
2    3
Name: c, dtype: int64

In [18]:
c * x

i  j
0  1        x[0,1]
   2        x[0,2]
1  0    2.0 x[1,0]
2  0    3.0 x[2,0]
   1    3.0 x[2,1]
dtype: object

Align the other way (by *names*):

$$
b_j x_{ij}
$$

In [19]:
b = pd.Series(index=pd.RangeIndex(3, name='j'), data=[1, 2, 3], name="b")
b

j
0    1
1    2
2    3
Name: b, dtype: int64

In [20]:
b * x

i  j
0  1    2.0 x[0,1]
   2    3.0 x[0,2]
1  0        x[1,0]
2  0        x[2,0]
   1    2.0 x[2,1]
dtype: object

Note this alignment is all performed by pandas, according to pandas rules

Slightly more complex:

$$
\sum_j b_j x_{ij} \quad \forall i \in I
$$

Here we are just taking the series `b * x` and applying the same groupby-aggregate operation as before:

In [21]:
(b * x).groupby("i").sum()

i
0    2.0 x[0,1] + 3.0 x[0,2]
1                     x[1,0]
2        x[2,0] + 2.0 x[2,1]
dtype: object

Note that we get a new series, with index $i$. This will be important shortly...

# Creating Constraints

- Using free functions


$$
\sum_j b_j x_{ij} \le 1 \quad \forall i \in I
$$

In [22]:
gppd.add_constrs(  
    model,
    (b * x).groupby("i").sum(),
    GRB.LESS_EQUAL,
    1.0,
    name="c1",
)

i
0    <gurobi.Constr c1[0]>
1    <gurobi.Constr c1[1]>
2    <gurobi.Constr c1[2]>
Name: c1, dtype: object

## Index alignment

- To use series on both sides, indexes must align

In [23]:
c = pd.Series(index=pd.RangeIndex(3, name="i"), data=[1, 2, 3])
c

i
0    1
1    2
2    3
dtype: int64

In [24]:
gppd.add_constrs(  
    model,
    (b * x).groupby("i").sum(),
    GRB.LESS_EQUAL,
    c,
    name="c1",
)

i
0    <gurobi.Constr c1[0]>
1    <gurobi.Constr c1[1]>
2    <gurobi.Constr c1[2]>
Name: c1, dtype: object

- Unaligned data, or missing data, is an error

In [25]:
gppd.add_constrs(  
    model,
    (b * x).groupby("i").sum(),
    GRB.LESS_EQUAL,
    pd.Series(index=pd.RangeIndex(4, name="i"), data=[1, 2, 3, 4]),
    name="c1",
)

KeyError: 'series must be aligned'

# Creating Constraints

- Using dataframe accessors

In [26]:
vars_and_constrs = variables.gppd.add_constrs(  
    model, "y + z <= 1", name="c1"
)
vars_and_constrs

p    w                    y                    z  \
i j                                                       
0 1  0.3  1.3  <gurobi.Var y[0,1]>  <gurobi.Var z[0,1]>   
  2  1.2  1.7  <gurobi.Var y[0,2]>  <gurobi.Var z[0,2]>   
1 0  0.7  1.4  <gurobi.Var y[1,0]>  <gurobi.Var z[1,0]>   
2 0  0.9  1.1  <gurobi.Var y[2,0]>  <gurobi.Var z[2,0]>   
  1  1.2  0.9  <gurobi.Var y[2,1]>  <gurobi.Var z[2,1]>   

                          c1  
i j                           
0 1  <gurobi.Constr c1[0,1]>  
  2  <gurobi.Constr c1[0,2]>  
1 0  <gurobi.Constr c1[1,0]>  
2 0  <gurobi.Constr c1[2,0]>  
  1  <gurobi.Constr c1[2,1]>

# Setting Objectives

Generally one objective (not an index-wise operation)

So we use `gurobipy` methods here

In [27]:
(x * data["p"]).sum()

<gurobi.LinExpr: 0.3 x[0,1] + 1.2 x[0,2] + 0.7 x[1,0] + 0.9 x[2,0] + 1.2 x[2,1]>

In [28]:
model.setObjective((x * data["p"]).sum(), sense=GRB.MAXIMIZE)

# Extracting Solutions

(Solve the model first)

In [29]:
model.optimize()
x.gppd.X  # Series accessor

i  j
0  1    0.000000
   2    0.333333
1  0    1.000000
2  0    1.000000
   1    0.000000
Name: x, dtype: float64

# Other Series accessor functionality

In [30]:
vars_and_constrs['c1']

i  j
0  1    <gurobi.Constr c1[0,1]>
   2    <gurobi.Constr c1[0,2]>
1  0    <gurobi.Constr c1[1,0]>
2  0    <gurobi.Constr c1[2,0]>
   1    <gurobi.Constr c1[2,1]>
Name: c1, dtype: object

In [31]:
vars_and_constrs['c1'].gppd.Slack

i  j
0  1    1.0
   2    1.0
1  0    1.0
2  0    1.0
   1    1.0
Name: c1, dtype: float64

In [32]:
expr = 2.0 * x + variables['y'] + 0.5
expr

i  j
0  1    0.5 + 2.0 x[0,1] + y[0,1]
   2    0.5 + 2.0 x[0,2] + y[0,2]
1  0    0.5 + 2.0 x[1,0] + y[1,0]
2  0    0.5 + 2.0 x[2,0] + y[2,0]
   1    0.5 + 2.0 x[2,1] + y[2,1]
dtype: object

In [33]:
expr.gppd.get_value()

i  j
0  1    0.500000
   2    1.166667
1  0    2.500000
2  0    2.500000
   1    0.500000
dtype: float64

# Example

[Project-Team Allocation](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/examples/projects.html). Focus on usage in practice. Key points:

- *Before* taking any modelling steps: prepare your data properly
    - Clearly define your model indexes, and align your dataframes to these indexes
    - Keep data reading & cleaning separate from model building
- Solutions are read back as numeric pandas data
    - Plug directly into the rest of pydata ecosystem for post-processing
    - Work pandonically with the results

- Show mathematical model
- Talk through the indexes and data

- Show the prepared dataframes
- Correct dtypes, no missing values, sparse index

- Walk through the code

# More examples are available

https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/examples.html

# Performance

[Dedicated page in the docs](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/performance.html). Key points:

- `gurobipy-pandas` won't magically make your model building code fast
- The API provides some structure to take *well-organized and prepared data* and help you build *clearly defined models* in the pandas style

# Final thoughts

- [Read the docs](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/index.html)
    - Check the examples for clean patterns to emulate
- [Check the repo](https://github.com/Gurobi/gurobipy-pandas)
    - Bugs? Feature request? Open an issue
- [Discuss!](https://support.gurobi.com/hc/en-us/community/topics/10373864542609-GitHub-Projects%3E)
    - For usage questions

# Thanks!

Questions?